In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


d:\Study\Learn ML\DL\env-tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd

# Base path
BASE_PATH = "../data/raw/"

# Load datasets
df  = pd.read_csv(BASE_PATH + "disaster_response_messages_training.csv")
dft = pd.read_csv(BASE_PATH + "disaster_response_messages_test.csv")
dfv = pd.read_csv(BASE_PATH + "disaster_response_messages_validation.csv")


C:\Users\ACER\AppData\Local\Temp\ipykernel_22352\4190944023.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv(BASE_PATH + "disaster_response_messages_training.csv")


In [7]:
df = pd.read_csv("../data/raw/disaster_response_messages_test.csv")
dft = pd.read_csv("../data/raw/disaster_response_messages_training.csv")
dfv = pd.read_csv("../data/raw/disaster_response_messages_validation.csv")


C:\Users\ACER\AppData\Local\Temp\ipykernel_22352\1968534392.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dft = pd.read_csv("../data/raw/disaster_response_messages_training.csv")


In [39]:
LABEL_MAP = {
    0: "real_disaster",
    1: "fake_casual",
    2: "noise_gibberish"
}

NUM_LABELS = 3

In [40]:
import re

def label_message(text):
    text = text.lower().strip()

    # ---------- 2️⃣ Noise / gibberish ----------
    # very short or random-looking text
    if len(text) < 6:
        return 2

    alpha_chars = sum(c.isalpha() for c in text)
    if alpha_chars / max(len(text), 1) < 0.6:
        return 2

    # ---------- 1️⃣ Fake / casual ----------
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "]+",
        flags=re.UNICODE
    )

    casual_words = ["hey", "hi", "lol", "bro", "dude", "haha", "jk"]

    if emoji_pattern.search(text):
        return 1

    if any(word in text for word in casual_words):
        return 1

    if re.search(r"(.)\1{3,}", text):  # fireeee
        return 1

    # ---------- 0️⃣ Real disaster ----------
    return 0


In [41]:
df["label"] = df["message"].apply(label_message)
df["label"].value_counts()


label
0    1568
1    1054
2       7
Name: count, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    df["message"].values,
    df["label"].values,         
    test_size=0.2,
    random_state=42,
    stratify=df["label"]       
)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))


Train size: 2103
Validation size: 526


In [43]:
print("Train label distribution:", np.bincount(y_train))
print("Val label distribution:", np.bincount(y_val))


Train label distribution: [1254  843    6]
Val label distribution: [314 211   1]


In [44]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)


In [45]:
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [65]:
def is_gibberish(text, min_len=6, min_alpha_ratio=0.6):
    text = text.strip()

    if len(text) < min_len:
        return True

    alpha_chars = sum(c.isalpha() for c in text)
    if alpha_chars / max(len(text), 1) < min_alpha_ratio:
        return True

    return False


In [69]:
train_dataset = FakeNewsDataset(
    X_train,
    y_train,
    tokenizer
)

val_dataset = FakeNewsDataset(
    X_val,
    y_val,
    tokenizer
)

In [70]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False
)


In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3  
)

model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [72]:
from torch.optim import AdamW

optimizer = AdamW(
    model.parameters(),
    lr=2e-5
)


In [73]:
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {avg_loss:.4f}")


Epoch 1/3 - Training Loss: 0.7188
Epoch 2/3 - Training Loss: 0.5246
Epoch 3/3 - Training Loss: 0.3012


In [74]:
SAVE_PATH = "fake_detection_distilbert"

model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("Model and tokenizer saved successfully.")

Model and tokenizer saved successfully.


In [75]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

infer_tokenizer = DistilBertTokenizerFast.from_pretrained(SAVE_PATH)
infer_model = DistilBertForSequenceClassification.from_pretrained(SAVE_PATH)

infer_model.to(device)
infer_model.eval()

print("Inference model loaded.")

Inference model loaded.


In [76]:
import re
import torch

LABEL_MAP = {
    0: "real_disaster",
    1: "fake_casual",
    2: "noise_gibberish"
}


def detect_fake(message):
    message_lower = message.lower().strip()
    
        # 0️⃣ Gibberish guard (ABSOLUTELY REQUIRED)
    if is_gibberish(message_lower):
        return {
            "is_fake": True,
            "class": "noise_gibberish",
            "reason": "gibberish_guard"
        }

    # ---------- Heuristic check (emoji) ----------
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "]+",
        flags=re.UNICODE
    )

    if emoji_pattern.search(message_lower):
        return {
            "is_fake": True,
            "class": "fake_casual",
            "reason": "emoji_detected"
        }

    # ---------- Model-based check ----------
    encoding = infer_tokenizer(
        message,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = infer_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred].item()

    predicted_class = LABEL_MAP[pred]

    return {
        "is_fake": pred != 0,                   
        "class": predicted_class,               
        "model_confidence": round(confidence, 3)
    }


In [83]:
def invalid_message(text):
    if len(text.strip()) < 6:
        return True

    alpha = sum(c.isalpha() for c in text)
    if alpha / len(text) < 0.6:
        return True

    if not any(word in text.lower() for word in [
        "fire", "flood", "earthquake", "storm", "help", "injured"
    ]):
        return True

    return False


In [77]:

for i, msg in enumerate(dft["message"].head(10)):
    result = detect_fake(msg)

    is_fake = result.get("is_fake")
    confidence = result.get("model_confidence", "N/A")

    print(f"\nSample {i+1}")
    print("Message:", msg)
    print("Fake:", is_fake)
    print("Confidence:", confidence)


Sample 1
Message: Weather update - a cold front from Cuba that could pass over Haiti
Fake: False
Confidence: 0.95

Sample 2
Message: Is the Hurricane over or is it not over
Fake: False
Confidence: 0.968

Sample 3
Message: says: west side of Haiti, rest of the country today and tonight
Fake: False
Confidence: 0.918

Sample 4
Message: Information about the National Palace-
Fake: False
Confidence: 0.923

Sample 5
Message: Storm at sacred heart of jesus
Fake: False
Confidence: 0.947

Sample 6
Message: Please, we need tents and water. We are in Silo, Thank you!
Fake: False
Confidence: 0.979

Sample 7
Message: I would like to receive the messages, thank you
Fake: False
Confidence: 0.98

Sample 8
Message: There's nothing to eat and water, we starving and thirsty.
Fake: True
Confidence: 0.98

Sample 9
Message: I am in Petionville. I need more information regarding 4636
Fake: False
Confidence: 0.975

Sample 10
Message: I am in Thomassin number 32, in the area named Pyron. I would like to have 

In [ ]:
custom_message = "jgviyivuifvuy"
print(detect_fake(custom_message))


{'is_fake': False, 'class': 'real_disaster', 'model_confidence': 0.753}


In [89]:
print(detect_fake("I met with an accident at the hills 10 peoples affected, need help"))

{'is_fake': False, 'class': 'real_disaster', 'model_confidence': 0.975}
